### Imports

In [1]:
import requests
import time

import os
import sys

import numpy as np
import pandas as pd 

sys.path.append("C:\\Users\\SHIRAM\\Documents\\WSB\\")
from generate_urls import get_all_tickers
from reddit_stuff import get_conn, get_sqlalchemy_engine, get_reddit_client_praw

import plotly.express as px
import plotly.graph_objects as go

from utils import batch
from wsb import Gather

from tqdm.auto import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

gather = Gather()

### Checkout local data

#### All submissions and comments

In [2]:
all_submisions_and_comments_df = pd.read_csv("../data/all_submissions_and_comments.csv", index_col=0)
all_submisions_and_comments_df.head()

,submission_id,comments
0,o1aztu,"{'data': ['h1zt5gv', 'h1ztc80', 'h1ztemi', 'h1..."
1,o1c8nx,"{'data': ['h2008md', 'h201mr6', 'h202942', 'h2..."
2,o18l7x,{'data': ['h1zvhr6']}
3,o1bbx1,"{'data': ['h1zunc1', 'h1zuuyv', 'h1zv22z', 'h1..."
4,o1b6hx,"{'data': ['h1ztx4f', 'h1zu2iy', 'h1zvw13', 'h1..."


#### all_saved_responses_before_sleep_.json

In [3]:
all_saved_responses_before_sleep_df = pd.read_json("../data/all_saved_responses_before_sleep.json")

In [4]:
all_saved_responses_before_sleep_df

,data
0,"[{'author': 'axon3010', 'author_created_utc': ..."
1,"[{'author': '[deleted]', 'author_flair_css_cla..."
2,"[{'author': 'poopy_donghoel', 'author_created_..."
3,"[{'author': 'Fiat_Tractor', 'author_created_ut..."
4,"[{'author': 'periklis1', 'author_created_utc':..."
...,...
495,"[{'all_awardings': [], 'allow_live_comments': ..."
496,[]
497,[]
498,[]


#### all_saved_responses_before_sleep_from_2020.json

In [5]:
all_saved_responses_before_sleep_from_2020_df = pd.read_json("../data/all_saved_responses_before_sleep_from_2020.json")

In [6]:
all_saved_responses_before_sleep_from_2020_df

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_patreon_flair,author_premium,...,author_flair_text_color,media,media_embed,secure_media,secure_media_embed,gallery_data,is_gallery,media_metadata,banned_by,edited
0,[],False,inever_lol,NaN,[],None,text,t2_lle7o,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,Winter-Click9229,NaN,[],None,text,t2_9hut4nyq,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,Jswimmin,NaN,[],None,text,t2_3xni2z21,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,Sir-Rupert-Everton,NaN,[],None,text,t2_77sl0ldk,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],False,Your_submissive_doll,NaN,[],None,text,t2_74i1t1b0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,[],False,Sudden-Wallaby424,NaN,[],None,text,t2_9cfiasro,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,[],False,Rizmo26,NaN,[],None,text,t2_682jx77k,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,[],False,am03ligu,NaN,[],None,text,t2_4n5gsiw6,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,[],False,Rockyturki,NaN,[],None,text,t2_a20yf17s,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Combine all_saved_responses_before_sleep_from_2020, all_responses_before_sleep, and all_submissions_and_comments

In [7]:
all_saved_responses_before_sleep_data = []
for i in range(len(all_saved_responses_before_sleep_df)):
    data = all_saved_responses_before_sleep_df.loc[0, "data"]
    all_saved_responses_before_sleep_data += data

df1 = pd.DataFrame.from_records(all_saved_responses_before_sleep_data)

columns_needed = [c for c in all_saved_responses_before_sleep_from_2020_df.columns if c in df1.columns]
columns_needed = [c for c in columns_needed if c not in ["preview", "secure_media_embed", "media", "secure_media", "media_embed"]]

df2 = all_saved_responses_before_sleep_from_2020_df[columns_needed]
df1 = df1[columns_needed]

all_submissions_combined = pd.concat([df1, df2])
all_submissions_combined = all_submissions_combined.drop_duplicates()

#### all_saved_responses_before_sleep_only_stickied.json

In [8]:
def read_and_process_only_stickied_subs():
    df = pd.read_json("../data/all_saved_responses_before_sleep_only_stickied.json")
    required_cols = ["created_utc", "id", "author", "author_id", "url", "title", "selftext", "stickied"]
    df = df[required_cols]

    reformat_cols = ["date", "id", "author", "url", "title", "selftext", "stickied"]
    df["date"] = pd.to_datetime(df["created_utc"], unit="s")
    df = df[reformat_cols]

    dates_present = df["date"].dt.date.unique().tolist()
    start_date, end_date = dates_present[0], dates_present[-1]
    all_dates = pd.date_range(start_date, end_date, freq="b").tolist()
    dates_absent = list(set(all_dates) - set(dates_present))
    return df, dates_absent

### Backfill data

In [9]:
def get_backfill_data(dates_absent) -> list:
    base_url = "https://api.pushshift.io/reddit/search/submission/?"
    before_after_dates = [(int((x - pd.Timedelta(days=1)).timestamp()), int((x + pd.Timedelta(days=1)).timestamp())) for x in dates_absent]

    # for all dates absent, find out if there are more stickied submissions, and merge them back into stickied_df at the end
    all_params = [{
            "subreddit": "wallstreetbets", 
            "stickied": "true", 
            "fields": "created_utc, id, author, url, title, selftext, stickied",
            "before": x[0],
            "after": x[1]
    } for x in before_after_dates]

    all_backfilled_data = []
    for param in tqdm(all_params): 
        response = requests.get(base_url, params=param)

        if response.status_code == 200: 
            data = response.json()
            if "data" in data.keys():
                all_backfilled_data.append(data["data"])

        time.sleep(1)

    return all_backfilled_data

In [11]:
stickied_df, dates_absent = read_and_process_only_stickied_subs()
# backfill_data = get_backfill_data(dates_absent)

In [ ]:
## If there's no data to be found, then just use stickied_df as base truth

In [13]:
stickied_df

,date,id,author,url,title,selftext,stickied
0,2017-08-26 10:46:14,6w4wra,WallStreetBot,https://www.reddit.com/r/wallstreetbets/commen...,New WSB Merch NOW Available,To commemorate current events. \n\n Mayweat...,True
1,2017-08-27 05:06:32,6waga2,bawse1,http://i.imgur.com/9njpbaw.png,/u/SUNEQ,,True
2,2017-09-14 12:30:12,701we2,SIThereAndThere,https://www.reddit.com/r/wallstreetbets/commen...,J.P. Morgan Early Look at the Market – Thurs 9...,J.P. Morgan Early Look at the Market – Thurs 9...,True
3,2017-09-20 03:53:30,718jpp,jartek,https://www.reddit.com/r/wallstreetbets/commen...,"Alright WSB, Mexico needs your help.",The Greater Mexico City Metropolitan Area was ...,True
4,2017-09-23 18:12:47,71zw49,SIThereAndThere,https://www.reddit.com/r/wallstreetbets/commen...,Goldman Sachs Dump: **PLEASE INVERSE THESE DOC...,Some shit from GS might be helpful to a few pe...,True
...,...,...,...,...,...,...,...
7074,2021-06-28 20:00:15,o9s4kb,OPINION_IS_UNPOPULAR,https://www.reddit.com/r/wallstreetbets/commen...,"What Are Your Moves Tomorrow, June 29, 2021",Your daily trading discussion thread. Please k...,True
7075,2021-06-29 10:00:14,oa56ol,OPINION_IS_UNPOPULAR,https://www.reddit.com/r/wallstreetbets/commen...,"Daily Discussion Thread for June 29, 2021",Your daily trading discussion thread. Please k...,True
7076,2021-06-29 10:00:49,oa570r,VisualMod,https://www.reddit.com/r/wallstreetbets/commen...,"Daily Popular Tickers Thread for June 29, 2021...",\nYour daily hype thread. Please keep the shit...,True
7077,2021-06-29 12:02:49,oa6vbq,VisualMod,https://www.reddit.com/r/wallstreetbets/commen...,"Daily Popular Tickers Thread for June 29, 2021...",\nYour daily hype thread. Please keep the shit...,True


### Cleanup all submissions and find missing dates

In [ ]:
all_submissions_combined["date"] = pd.to_datetime(all_submissions_combined["created_utc"], unit="s")
all_submissions_combined = all_submissions_combined[stickied_df.columns.tolist()]
submissions_df = pd.concat([stickied_df, all_submissions_combined])
submissions_df = submissions_df.drop_duplicates()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax = submissions_df.groupby(submissions_df["date"].dt.date).apply(lambda x: x["id"].nunique()).plot(ax=ax)
ax = ax.vlines(x=dates_absent, ymin=24, ymax=25, colors='red',  lw=1, label='vline_multiple')

### Cut down wsb_comments_temp

In [ ]:
comments_columns = ["created_utc", "retrieved_on", "id", "parent_id", "link_id", "author", "author_fullname", "submission_id", "subreddit_name_prefixed", "body", "permalink"]

### Populate local database